__2. Data__

For the purpose of our analysis, we obtained the data of all Toronto neighbourhoods using the most common and easily accesible source, i.e. wikipedia website. We formatted and cleaned the data for our purposes, per the below.

Let's obtain data of all postal codes in Canada staring with letter 'M' (i.e. including Toronto neighbourhoods).

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib.request
import numpy as np

%config IPCompleter.greedy = True

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
table_from_top = 1
wikipedia_page = 'List_of_postal_codes_of_Canada'
trace = False

In [3]:
wikipedia_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'.format(wikipedia_page)
page = requests.get(wikipedia_url)
soup = BeautifulSoup(page.content, 'lxml')
tables = soup.find_all('table', {'class': 'wikitable'})
table = tables[table_from_top - 1]

In [4]:
feature_names = []

header_row = table.find('tr')
for header in header_row.find_all('th'):
    feature_name = ' '.join(header.find_all(text=True))
    feature_name.replace('\n', '')
    feature_names.append(feature_name)

'Postal Code'

'Borough'

'Neighbourhood'

In [5]:
def has_coords(tag):
    if tag.has_attr('class'):
        if tag['class'][0] == 'latitude' or tag['class'][0] == 'longitude':
            return True
    return False

def get_coords(child):
    coords = []
    for coord in child.find_all(has_coords):
        coords.append(coord.string)
    if coords:
        if trace:
            return 'C = {}'.format(' '.join(coords))
        else:
            return ' '.join(coords)
    else:
        return ''

samples = []
sample_rows = table.find_all('tr')[1:]
for sample_row in sample_rows:
    features = []
    for feature_col in sample_row.find_all('td'):
        feature_value = ''
        text = feature_col.string
        if text:
            if trace:
                features.append('T = {}'.format(text))
            else:
                features.append(text)
            continue
        
        for child in feature_col.children:
            if child.name == 'span':
                if child.has_attr('class'):
                    if child['class'] == 'display:none':
                        continue
                if child.find_all(has_coords):
                    feature_value = get_coords(child)
                    if feature_value:
                        break
                    else:
                        continue
            if child.name == 'sup':
                continue
            if child.name == 'a':
                if child.string[0] == '[':
                    continue            
            if child.name == 'a':
                if trace:
                    feature_value = 'A = {}'.format(child.string)
                else:
                    feature_value = child.string
                break
            if child.name == 'font':
                if trace:
                    feature_value = 'F = {}'.format(child.string)
                else:
                    feature_value = child.string
                break
            try:
                # feature_value = '' for any tags not covered above
                content = child.contents
            except AttributeError:
                # Handle whitespace between child tags, treated as a child string
                if child.isspace():
                    continue
                if trace:
                    feature_value = 'E = {}'.format(child)
                else:
                    feature_value = child
                break
        features.append(feature_value)
    samples.append(dict(zip(feature_names, features)))

In [6]:
canada = pd.DataFrame(samples)
canada.head()

,Postal Code\n,Borough\n,Neighbourhood\n
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


We will format the table so we can perform further analysis.

In [7]:
canada.rename(columns={'Postal Code\n': 'Postal Code', 'Borough\n': 'Borough', 'Neighbourhood\n': 'Neighbourhood'}, inplace=True)
canada.head()

,Postal Code,Borough,Neighbourhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [8]:
canada['Postal Code'] = canada['Postal Code'].str.replace(r'\n', '')
canada['Borough'] = canada['Borough'].str.replace(r'\n', '')
canada['Neighbourhood'] = canada['Neighbourhood'].str.replace(r'\n', '')
canada.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Now, we will choose only Toronto neighbourhoods, filtering the data for 'Toronto' in Borough column.

In [9]:
toronto = canada[canada['Borough'].str.contains('Toronto')]
toronto=toronto.reset_index(drop=True)
toronto

,Postal Code,Borough,Neighbourhood
0,M5A,Downtown Toronto,"Regent Park, Harbourfront"
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
2,M5B,Downtown Toronto,"Garden District, Ryerson"
3,M5C,Downtown Toronto,St. James Town
4,M4E,East Toronto,The Beaches
5,M5E,Downtown Toronto,Berczy Park
6,M5G,Downtown Toronto,Central Bay Street
7,M6G,Downtown Toronto,Christie
8,M5H,Downtown Toronto,"Richmond, Adelaide, King"
9,M6H,West Toronto,"Dufferin, Dovercourt Village"


In [10]:
toronto.shape

(39, 3)

In [11]:
test=toronto.groupby('Borough').count()
test

,Postal Code,Neighbourhood
Borough,,
Central Toronto,9,9
Downtown Toronto,19,19
East Toronto,5,5
West Toronto,6,6


Now that we obtain the list of potential neighbourhoods in Toronto, let's add coordinates using geocoder function.

In [12]:
!pip install geocoder

In [13]:
import geocoder

In [14]:
Postal_Code = toronto['Postal Code']

In [15]:
latitude=[]
longitude=[]
for code in toronto['Postal Code']:
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    print(code, g.latlng)
    while (g.latlng is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
        print(code, g.latlng)
    latlng = g.latlng
    latitude.append(latlng[0])
    longitude.append(latlng[1])

M5A [43.65512000000007, -79.36263999999994]
M7A [43.66253000000006, -79.39187999999996]
M5B [43.65739000000008, -79.37803999999994]
M5C [43.65215000000006, -79.37586999999996]
M4E [43.67709000000008, -79.29546999999997]
M5E [43.64536000000004, -79.37305999999995]
M5G [43.65609000000006, -79.38492999999994]
M6G [43.668690000000026, -79.42070999999999]
M5H [43.64970000000005, -79.38257999999996]
M6H [43.665050000000065, -79.43890999999996]
M5J [43.64285000000007, -79.38075999999995]
M6J [43.64848000000006, -79.41773999999998]
M4K [43.68375000000003, -79.35511999999994]
M5K [43.64710000000008, -79.38152999999994]
M6K [43.639410000000055, -79.42675999999994]
M4L [43.667970000000025, -79.31466999999998]
M5L [43.64840000000004, -79.37913999999995]
M4M [43.66213000000005, -79.33496999999994]
M4N [43.72843000000006, -79.38712999999996]
M5N [43.71208000000007, -79.41847999999999]
M4P [43.71276000000006, -79.38850999999994]
M5P [43.69479000000007, -79.41439999999994]
M6P [43.659730000000025, -79

In [16]:
toronto['Latitude'] = latitude
toronto['Longitude'] = longitude
toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804
3,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587
4,M4E,East Toronto,The Beaches,43.67709,-79.29547


Let's visualize the data. For that ourpose, we will use folium and json packages.

In [17]:
!pip install folium

In [18]:
import folium
import json
from geopy.geocoders import Nominatim 

In [19]:
address = 'Toronto'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [20]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Now, let's leverage Foursquare API data to obtain list and locations of all venues in Toronto.

In [21]:
CLIENT_ID = 'FUD1ZSKEBF30D3424K0LBAZCUWDSLSQQPLYFCAVQ3EQSUJR2' # your Foursquare ID
CLIENT_SECRET = '0PXRI5XYRPT32PR4YDHC2AKNGTX5O0MX42OOPXN2XDZIB2WU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FUD1ZSKEBF30D3424K0LBAZCUWDSLSQQPLYFCAVQ3EQSUJR2
CLIENT_SECRET:0PXRI5XYRPT32PR4YDHC2AKNGTX5O0MX42OOPXN2XDZIB2WU


We will leverage below previously created function to obtain all venues located in Toronto.

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
toronto_venues = getNearbyVenues(names=toronto['Neighbourhood'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )


Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


After obtaining the list of all venues in Toronto, we will filter the results to include places relevant to our business problem. Since the investor is interested in opening new location for his gym chain, we will include venues that contain __'Gym'__ in the 'Venue category' for the purpose of our analysis.<br>
Additionally, for the purpose of our analysis, we will include places which might potentially impact attendance at the gym, i.e. places which are considered natural competitors of gyms, that is any venues containing __'Park', 'Sport', 'Pool', 'Stadium', 'Studio'__.

In [24]:
pd.set_option('display.max_rows', None)
categories=toronto_venues.groupby(['Venue Category']).count()
categories

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Accessories Store,1,1,1,1,1,1
Adult Boutique,2,2,2,2,2,2
American Restaurant,25,25,25,25,25,25
Antique Shop,2,2,2,2,2,2
Aquarium,2,2,2,2,2,2
Art Gallery,13,13,13,13,13,13
Art Museum,1,1,1,1,1,1
Arts & Crafts Store,5,5,5,5,5,5
Asian Restaurant,19,19,19,19,19,19


In [25]:
pd.set_option('display.max_rows', 10)

In [26]:
toronto_venues_gym=toronto_venues[toronto_venues['Venue Category'].str.contains('Pool|Gym|Park|Sport|Stadium|Studio', na=False)]
toronto_venues_gym=toronto_venues_gym.reset_index(drop=True)
toronto_venues_gym.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65512,-79.36264,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
1,"Regent Park, Harbourfront",43.65512,-79.36264,The Extension Room,43.653313,-79.359725,Gym / Fitness Center
2,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188,Queen's Park,43.663946,-79.392180,Park
3,"Garden District, Ryerson",43.65739,-79.37804,306 Yonge Street - Jordan Store,43.656495,-79.381015,Sporting Goods Shop
4,"Garden District, Ryerson",43.65739,-79.37804,Hard Candy Fitness,43.659556,-79.382440,Gym / Fitness Center


Let's visualize the data.

In [27]:
map_toronto_venues_gym = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, neighbourhood in zip(toronto_venues_gym['Venue Latitude'], toronto_venues_gym['Venue Longitude'], toronto_venues_gym['Neighbourhood']):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_venues_gym)  
    
map_toronto_venues_gym

After a quick glance at the map we can see there are potentially places that are better suited for a new gym than other. We will perform formal data analysis to be sure. Let's do a little formatting on our table to include only data that we will be using for further analysis.

In [28]:
toronto_gym=toronto_venues_gym[['Neighbourhood', 'Venue', 'Venue Category']]
toronto_gym

,Neighbourhood,Venue,Venue Category
0,"Regent Park, Harbourfront",The Yoga Lounge,Yoga Studio
1,"Regent Park, Harbourfront",The Extension Room,Gym / Fitness Center
2,"Queen's Park, Ontario Provincial Government",Queen's Park,Park
3,"Garden District, Ryerson",306 Yonge Street - Jordan Store,Sporting Goods Shop
4,"Garden District, Ryerson",Hard Candy Fitness,Gym / Fitness Center
...,...,...,...
106,"Business reply mail Processing Centre, South C...",Cardio-Go,Gym
107,"Business reply mail Processing Centre, South C...",Equinox Bay Street,Gym
108,"Business reply mail Processing Centre, South C...",The Cambridge Club,Gym
109,"Business reply mail Processing Centre, South C...",Osgoode Hall Park,Park
